In [264]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# Conversione documenti ("Juventus" = "home team" or "away team")
In questo script voglio leggere i documenti e dato che una squadra può avere diversi soprannomi, poi per la classificazione risulta complicato andare a capire di chi si sta parlando. Per questo motivo vado a considerare tutti i soprannomi utilizzati nella lega rispetto alle squadre e in ciascun documento (in questo caso lo faccio su un solo documento) vado a sostituire il nome della squadra e tutti i suoi soprannomi con "home team" o "away team".
I sinonimi vengono presi da un dizionario.

Ho due problemi: 
- ci sono squadre come "Hellas Verona" che può anche essere chiamato "Verona" quindi devo stare attento nel dizionario a posizionare "Hellas Verona" prima di "Verona", altrimenti se posizionassi al contrario, quando nel documento sostituisco con ad esempio "home team", risulta "Hellas home team", invece anche Hellas deve sparire
- un problema simile a quello descritto sopra è per le abbreviazioni come "Juve" e "Juventus" quindi per non trovarci in una situazione simile "home teamntus" dobbiamo mettere la stringa completa prima dell'abbreviata

Un altro problema è legato alla stadio, perché qui il nome di alcuni stadi come "Olimpico di Torino" viene presa la parola Torino e sostituita. Questo dovrebbe succedere o no? Potrei sostituire il nome dello stadio con "home stadium" o "away stadium"

In [265]:
import urllib.request
import json
import spacy
nlp = spacy.load("en_core_web_sm")

tokenize = lambda text: [x.lemma_.lower() for x in text if x.pos_ in ['NOUN', 'PROPN', 'VERB', 'ADJ', 'ADV']]

In [266]:
dataframe = pd.read_csv("description_predictions.csv", index_col=0)

In [267]:
with open("sinonimi.json") as jsonFile:
    jsonObject = json.load(jsonFile)
    jsonFile.close()

In [268]:
descriptions = dataframe.iloc[2:] #cambio il numero del record. In questo caso prendo il dataset dal record 2 in poi
#documento utile 101 
descriptions = descriptions.iloc[:1] #prendo una singola partita

Estraggo le squadre, perché nel testo ci sono i soprannomi delle squadre e quindi non essendo conosciuti dal vocabolario, li sostituisco con il nome della relativa squadra

In [269]:
h_team, a_team, description = descriptions.h_team.values[0], descriptions.a_team.values[0], descriptions.description.values[0]
print(h_team, a_team, "\n"+description)

syn = {}
for key in jsonObject.keys():
    if (h_team in key) or (key in h_team):
        syn['home team'] = jsonObject[key] 
        
    if (a_team in key) or (key in a_team):
        syn['away team'] = jsonObject[key] 

Hellas Verona Sassuolo 
Verona and Sassuolo take on each other in a highly-anticipated Serie A game at Stadio Marc’Antonio Bentegodi on Saturday evening. The Mastini have established themselves as a top-table side in the Italian top flight, but the game against Sassuolo will be anything but a walk in the park for the hosts. The club chiefs did well to keep hold of key players in the summer transfer window, but keep in mind that both Lasagna and Faraoni are likely to miss the season opener. 

The Neroverdi, on the other hand, were producing fine displays in pre-season, with the team beating both Parma and Vicenza, while sharing the spoils with Lazio. Considering that Sassuolo are likely to stick to their attacking style of play, both teams to score betting option should be considered. Centre-back Marlon is no longer part of the team, while Kyriakopoulos is banned for the match. Obiang is yet another absentee in the away team.


In [270]:
print("Game: "+h_team+" - "+a_team)
print("ORIGINAL DOCUMENT\n"+description)
description = description.lower()
for key in syn.keys():
    for val in syn[key]:
        description = description.replace(val.lower(), key)

print("\n\nCHANGED DOCUMENT\n"+description)

Game: Hellas Verona - Sassuolo
ORIGINAL DOCUMENT
Verona and Sassuolo take on each other in a highly-anticipated Serie A game at Stadio Marc’Antonio Bentegodi on Saturday evening. The Mastini have established themselves as a top-table side in the Italian top flight, but the game against Sassuolo will be anything but a walk in the park for the hosts. The club chiefs did well to keep hold of key players in the summer transfer window, but keep in mind that both Lasagna and Faraoni are likely to miss the season opener. 

The Neroverdi, on the other hand, were producing fine displays in pre-season, with the team beating both Parma and Vicenza, while sharing the spoils with Lazio. Considering that Sassuolo are likely to stick to their attacking style of play, both teams to score betting option should be considered. Centre-back Marlon is no longer part of the team, while Kyriakopoulos is banned for the match. Obiang is yet another absentee in the away team.


CHANGED DOCUMENT
home team and awa